In [1]:
import pandas as pd                                     
import numpy as np
import os                                               
import matplotlib.pyplot as plt 
import scipy.stats.mstats as ssm
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn import preprocessing
import random

%matplotlib inline

/Users/nachoaguadoc/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('CrowdstormingDataJuly1st.csv')
data = data.dropna(axis=0, subset= ['position'])

In [3]:
data.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


In [4]:
data.set_index(data.columns[0], inplace = True)
data

,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
playerShort,,,,,,,,,,,,,,,,,,,,,
lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,0,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,0,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,0,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
anders-lindegaard,Anders Lindegaard,Manchester United,England,13.04.1984,193.0,80.0,Goalkeeper,1,0,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
andreas-beck,Andreas Beck,1899 Hoffenheim,Germany,13.03.1987,180.0,70.0,Right Fullback,1,1,0,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


In [5]:
# Club Dummie Variable 
# Country Dummie Variable
# height
# weight
# position Dummie Variable
# SkinTone Label

def skinLabels(rater):
    if  rater <= 0.5:
        rater = 0
    else:
        rater = 1
    return rater

In [6]:
label_encoder = preprocessing.LabelEncoder()
players_wr = data[['player', 'height', 'weight']].copy()
players_wr['club'] = label_encoder.fit_transform(data['club'])
players_wr['leagueCountry'] = label_encoder.fit_transform(data['leagueCountry'])
players_wr['position'] = label_encoder.fit_transform(data['position'])
players_wr['skincolor'] = [skinLabels(x) for x in ((data['rater1']+data['rater2'])/2).values]
players_wr = players_wr.dropna(axis = 0, subset=['skincolor'])

In [7]:
players_wr.head()
players_wr.shape

(128302, 7)

In [8]:
#new_players_wr = players_wr.dropna(axis=0, subset= ['height','weight','club', 'leagueCountry'])
new_players_wr = players_wr.drop_duplicates(['player'], keep='first')
new_players_wr
new_players_wr['height'] = new_players_wr['height'].fillna(np.mean(new_players_wr['height'].dropna()))
new_players_wr['weight'] = new_players_wr['weight'].fillna(np.mean(new_players_wr['weight'].dropna()))
new_players_wr['club'] = new_players_wr['club'].fillna(np.mean(new_players_wr['club'].dropna()))
new_players_wr['leagueCountry'] = new_players_wr['leagueCountry'].fillna(np.mean(new_players_wr['leagueCountry'].dropna()))
new_players_wr['position'] = new_players_wr['position'].fillna(np.mean(new_players_wr['position'].dropna()))


new_players_wr['height'] = (new_players_wr['height']-new_players_wr['height'].mean())/new_players_wr['height'].std()
new_players_wr['weight'] = (new_players_wr['weight']-new_players_wr['weight'].mean())/new_players_wr['weight'].std()


/Users/nachoaguadoc/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nachoaguadoc/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nachoaguadoc/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

In [52]:
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = new_players_wr.drop(['skincolor'], axis=1)
Y = new_players_wr['skincolor']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=666)

k_fold = KFold(x_train.shape[0], n_folds = 20, shuffle=True)
scores = []
importances = np.zeros(5)
for train_indices, test_indices in k_fold:
    rf = RandomForestClassifier(n_estimators=20)
    rf.fit(x_train.iloc[train_indices][['height','weight','club', 'leagueCountry','position']], y_train[train_indices])
    score = rf.score(x_train.iloc[test_indices][['height','weight','club', 'leagueCountry','position']], y_train[test_indices])
    importance = np.asarray(rf.feature_importances_)
    scores.append(score)
    importances = np.add(importances,importance)

print(np.array(scores).mean(), np.array(scores).std())

0.739265593561 0.045018396379


In [59]:
rf = RandomForestClassifier(n_estimators=20)
rf.fit(x_train[['height','weight','club', 'leagueCountry','position']], y_train)
print(rf.score(x_test[['height','weight','club', 'leagueCountry','position']], y_test))

0.749003984064


In [67]:
rf.decision_path(x_train.drop('player',1))

(<1419x12666 sparse matrix of type '<class 'numpy.int64'>'
 	with 323084 stored elements in Compressed Sparse Row format>,
 array([    0,   695,  1326,  1979,  2610,  3211,  3894,  4527,  5200,
         5815,  6476,  7049,  7660,  8299,  8898,  9501, 10116, 10801,
        11448, 12035, 12666]))